# Composable Objects

이 노트북에서는 여러 개체를 단일 최상위 인덱스로 결합하는 방법을 보여줍니다. 

이 접근 방식은 다음을 가리키는 obj 필드를 사용하여 IndexNode 객체를 설정함으로써 작동합니다.


- query engine
- retriever
- query pipeline
- another node!


```python
object = IndexNode(index_id="my_object", obj=query_engine, text="some text about this object")
```

## Setup Langfuse

In [1]:
from llama_index.core import Settings
from llama_index.core.callbacks import CallbackManager
from langfuse.llama_index import LlamaIndexCallbackHandler

langfuse_callback_handler = LlamaIndexCallbackHandler(
    public_key="pk-lf-29339a8f-8a05-4d10-9a0a-a4718f79a53d",
    secret_key="sk-lf-d1f94511-893b-456e-973e-f57b44d50a30",
    host="https://cloud.langfuse.com"
)
Settings.callback_manager = CallbackManager([langfuse_callback_handler])

## Data Setup

In [19]:
"""Load from python file"""

import sys

# 절대 경로를 sys.path에 추가
sys.path.append('/Users/heewungsong/Experiment/Visa_Rag/study/llama-index/assets/visa_docs/')

# 모듈 임포트
from LlamaIndexVisaDocument import LlamaIndexE9VisaDocuments

docs = LlamaIndexE9VisaDocuments

In [14]:
"""Load from pdf file"""

from llama_index.core import SimpleDirectoryReader
from llama_index.readers.file import PyMuPDFReader

visa_pdf_file_path = "../assets/visa_docs/E-9 Visa Guide_한국어.pdf"
visa_docs = PyMuPDFReader().load_data(file_path=visa_pdf_file_path, metadata=True)

faq_pdf_file_path = "../assets/visa_docs/E-9 FAQ_한국어.pdf"
faq_docs = PyMuPDFReader().load_data(file_path=faq_pdf_file_path, metadata=True)

In [4]:
# from llama_index.core.node_parser import TokenTextSplitter, SentenceSplitter, SentenceWindowNodeParser

# text_splitter = SentenceSplitter()

# node_parser = SentenceWindowNodeParser.from_defaults(
#     window_size=3,
#     window_metadata_key="window",
#     original_text_metadata_key="original_text",
# )

# visa_guide_sentence_nodes = node_parser.get_nodes_from_documents(docs)
# visa_guide_nodes = text_splitter.get_nodes_from_documents(docs)

# print(f"Count of sentence_nodes: {len(visa_guide_sentence_nodes)}")
# print(f"Count of base_nodes: {len(visa_guide_nodes)}")

Count of sentence_nodes: 26
Count of base_nodes: 24


In [5]:
# from llama_index.core import VectorStoreIndex
# from llama_index.core.postprocessor import MetadataReplacementPostProcessor

# sentence_index = VectorStoreIndex(visa_guide_sentence_nodes)

# query_engine = sentence_index.as_query_engine(
#     similarity_top_k=2,
#     # the target key defaults to `window` to match the node_parser's default
#     node_postprocessors=[
#         MetadataReplacementPostProcessor(target_metadata_key="window")
#     ],
# )


In [ ]:
# window_response = query_engine.query(
#     "비전문취업 비자의 일반적인 허용 업종 범위는 어디까지인가요?"
# )
# print(window_response)

## Retriever Setup

In [25]:

from llama_index.core.node_parser import TokenTextSplitter

nodes = TokenTextSplitter(
    chunk_size=1024, chunk_overlap=128
).get_nodes_from_documents(visa_docs + faq_docs)

In [26]:
len(nodes)

72

In [27]:
from llama_index.core.storage.docstore import SimpleDocumentStore

docstore = SimpleDocumentStore()
docstore.add_documents(nodes)

In [28]:
from llama_index.core import VectorStoreIndex
from llama_index.retrievers.bm25 import BM25Retriever

index = VectorStoreIndex(nodes=nodes)
vector_retriever = index.as_retriever(similarity_top_k=2)
bm25_retriever = BM25Retriever.from_defaults(
    docstore=docstore, similarity_top_k=2
)

In [29]:
from llama_index.core.schema import IndexNode

vector_obj = IndexNode(
    index_id="vector", obj=vector_retriever, text="Vector Retriever"
)
bm25_obj = IndexNode(
    index_id="bm25", obj=bm25_retriever, text="BM25 Retriever"
)

In [30]:
from llama_index.core import SummaryIndex

summary_index = SummaryIndex(objects=[vector_obj, bm25_obj])

In [31]:
query_engine = summary_index.as_query_engine(
    response_mode="tree_summarize", verbose=True
)

In [32]:
response = await query_engine.aquery(
    "비전문취업 비자(E-9)로 취업 가능한 업종과 해당 업종코드는 무엇인가요?"
)
print(str(response))

Retrieval entering vector: VectorIndexRetriever


Retrieval entering bm25: BM25Retriever


In [34]:
response = await query_engine.aquery("E9은 농업이나 건설업 중에 어느쪽이 취업이 더 빨라요?")
print(str(response))

Retrieval entering vector: VectorIndexRetriever


Retrieval entering bm25: BM25Retriever
E9 비자를 받아 한국에서 취업하려면 제조업, 건설업, 농축산업, 어업 등 허용된 업종에서 일할 수 있습니다. 구체적인 취업 속도에 대한 정보는 제공되지 않았습니다.


In [35]:
response = await query_engine.aquery("중학교 졸업증만 있는데 한국에서 일할 수 있어요?")
print(str(response))

Retrieval entering vector: VectorIndexRetriever


Retrieval entering bm25: BM25Retriever
Yes, it is possible to work in Korea with just a middle school graduation certificate. The E-9 visa for non-professional employment does not specifically require a certain level of education. Foreign workers who meet the employment requirements under the law can be eligible for this visa. However, to obtain the E-9 visa, one must receive a work permit and submit documents such as a criminal record certificate and a health check certificate. Additionally, finding a suitable employer in accordance with the employment permit system and ensuring that the employer meets the conditions for hiring foreign workers are necessary steps.


In [38]:
response = await query_engine.aquery("한국비자에 대해 알아보고 있는데 여자이면 가기 어려운가요?")
print(str(response))

Retrieval entering vector: VectorIndexRetriever


Retrieval entering bm25: BM25Retriever
성별이 여성이라는 이유만으로 한국에서 E-9 비자 발급이 어렵다고 할 수는 없습니다. E-9 비자의 발급은 주로 해당 업종에 대한 수요, 고용주의 요건 충족, 그리고 외국인 근로자의 법적 요건 충족 등에 초점을 맞추고 있습니다. 특정 업종이나 직무에 따라 신체적 조건이나 성별에 대한 요구사항이 있을 수 있으며, 이는 고용주가 결정하는 사항입니다. 따라서, 여성이 한국에서 E-9 비자를 받는 것이 어려운지는 해당 업종이나 직무의 요구사항에 따라 다를 수 있습니다.


In [39]:
for idx, node in enumerate(response.source_nodes):
    print(f"---------------- Source: {node.metadata['source']} ----------------")
    print(node)
    print("--------------------------------\n\n")
    # print(node.score)

---------------- Source: 19 ----------------
Node ID: 9ba2f5fe-2422-4741-8494-0a2efeff8039
Text: 06년생여자예요. 공부마치고한국에서일하려고해요. 그런데 몸이작고153cm, 37킬로라서한국에서E9비자로근무
가능할까요? 여자이면어렵다고해서걱정돼요 E-9 비자는한국에서비전문취업을위한체류자격을부여하는비자입니다. 비자발급과
관련하여신체적조건(키와몸무게)이직접적인제한요소로명시되어있지는않습니다. E-9 비자의발급요건은주로해당업종에대한수요,
고용주의요건충족, 그리고외국인 근로자의법적요건충족등에초점을맞추고있습니다. 성별이여성이라는이유만으로E-9
비자발급이어렵다고할수는없습니다. 다만, 특정 업종이나직무에따라신체적조건이나성별에대한요구사항이있을수있으며, 이는
고용주가결정하는사항입니다. 예를들어, 무거운물건을자주들어야하는직무나 특정...
Score:  0.838

--------------------------------


---------------- Source: 37 ----------------
Node ID: 439ad00d-9e27-43b1-9ca6-12de5eb68325
Text: 여자친구가필리핀사람인데E9 비자로4년10개월한국에있다가 코로나로1년연장할수있다고해서했거든요. 총5년10개월...
이제 슬슬비자가만기가되서필리핀가서시험쳐서또올수있다고 하더라구요. 4년10개월인가 비자 더 받을수있다고 하던데..
한국비자최대10년으로알고있어요. 근데이렇게코로나로 연장했는경우도5년받을수있나요? 아니면1년빼고4년만 비자대나요?
시험칠때자격같은건이상없을까요? ● 비전문취업(E-9) 비자를가진외국인근로자의경우, 기본적으로체류기간은
입국일로부터최대3년입니다. 그후, 고용주가재고용을신청하면최대1년 10개월의체류기간연장이가능합니다. 따라서,
일반적인상황에서는최대4년 10개월까지한국에서근무할수있습니다. ● 코로나19...
Score:  0.830

--------------------------------


----

## Save Store

In [37]:
# save our docstore nodes for bm25

save_json_path = "./stores/composabel_visa_docs+faq_docs.json"
docstore.persist(save_json_path)

## Load

In [ ]:

from llama_index.core.storage.docstore import SimpleDocumentStore

docstore = SimpleDocumentStore.from_persist_path(save_json_path)


index = VectorStoreIndex.from_vector_store(vector_store)
vector_retriever = index.as_retriever(similarity_top_k=2)

bm25_retriever = BM25Retriever.from_defaults(
    docstore=docstore, similarity_top_k=2
)